# Import libraries

In [1]:
import pandas as pd
from scipy import stats
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import preprocessing
from scipy.stats import mannwhitneyu
from scipy.stats import shapiro

# Intialization

In [2]:
# Constants 
NUM_AI_STATEMENTS = 10
NUM_HUMAN_STATEMENTS = 10

In [3]:
# Function Definitions

# Defining a function for findig the sequence ID
def findStatementSequence(sequence_of_answers, target_statement):
    seperator_positions = [i for i, letter in enumerate(sequence_of_answers) if letter == '|']
    position_of_statement = sequence_of_answers.find('|' + target_statement)
    sequence_id = seperator_positions.index(position_of_statement)
    
    return sequence_id

# Renaming the sequence lables imported from Qualrtics

def renamingSequenceString(sequence_of_answers):
    sequence_of_answers = sequence_of_answers.replace('|Bing1|', '|AI01|') \
    .replace('|Bing2|', '|AI02|').replace('|Bing3|', '|AI03|') \
    .replace('|Bing4|', '|AI04|').replace('|Bing5|', '|AI05|').replace('|Bing7|', '|AI06|') \
    .replace('|Bing7|', '|AI06|').replace('|Bing8|', '|AI07|').replace('|Bing9|', '|AI08|') \
    .replace('|Bing10|', '|AI09|').replace('|Bing11|', '|AI10|').replace('|Human1|', '|Hum01|') \
    .replace('|Human2|', '|Hum02|').replace( '|Human3|', '|Hum03|').replace('|Human4|', '|Hum04|') \
    .replace('|Human5|', '|Hum05|').replace('|Human6|', '|Hum06|').replace('|Human7|', '|Hum07|') \
    .replace('|Human8|', '|Hum08|').replace('|human9|', '|Hum09|').replace('|Human10|', '|Hum10|')
    return sequence_of_answers

# Raw data filtering

In [4]:
#set the data directory
data_dir = '/Users/admin/Library/CloudStorage/OneDrive-UniversitédeLausanne/Fake_news/Code/Data/'

In [5]:
#load the raw data
raw_df = pd.read_csv(data_dir + 'fake_news_final_data.csv')
raw_df

,StartDate,EndDate,Status,IPAddress,Progress,Duration (in seconds),Finished,RecordedDate,ResponseId,RecipientLastName,...,Religious,Religious_DO,Q224_First Click,Q224_Last Click,Q224_Page Submit,Q224_Click Count,rid,gc,FL_12_DO,CRT_DO
0,Start Date,End Date,Response Type,IP Address,Progress,Duration (in seconds),Finished,Recorded Date,Response ID,Recipient Last Name,...,Would you say you are:,Would you say you are: - Display Order,Timing - First Click,Timing - Last Click,Timing - Page Submit,Timing - Click Count,rid,gc,FL_12 - Block Randomizer - Display Order,CRT - Display Order
1,"{""ImportId"":""startDate"",""timeZone"":""America/De...","{""ImportId"":""endDate"",""timeZone"":""America/Denv...","{""ImportId"":""status""}","{""ImportId"":""ipAddress""}","{""ImportId"":""progress""}","{""ImportId"":""duration""}","{""ImportId"":""finished""}","{""ImportId"":""recordedDate"",""timeZone"":""America...","{""ImportId"":""_recordId""}","{""ImportId"":""recipientLastName""}",...,"{""ImportId"":""QID126""}","{""ImportId"":""QID126_DO""}","{""ImportId"":""QID236_FIRST_CLICK""}","{""ImportId"":""QID236_LAST_CLICK""}","{""ImportId"":""QID236_PAGE_SUBMIT""}","{""ImportId"":""QID236_CLICK_COUNT""}","{""ImportId"":""rid""}","{""ImportId"":""gc""}","{""ImportId"":""FL_12_DO""}","{""ImportId"":""BL_9LiFt4WQPmKdK0m_DO""}"
2,2023-06-09 06:30:57,2023-06-09 06:31:21,0,172.58.128.72,100,23,1,2023-06-09 06:31:22,R_11je2fa572WqFmx,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,64831b6b-9735-691a-bdf7-e33d56c5b9df,2,NaN,NaN
3,2023-06-09 06:31:05,2023-06-09 06:31:27,0,50.29.230.143,100,22,1,2023-06-09 06:31:28,R_32Watpb2YtUTkH7,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,64831b66-b468-a5ab-b86b-764acdf1cabf,2,NaN,NaN
4,2023-06-09 06:31:18,2023-06-09 06:31:48,0,24.198.65.80,100,30,1,2023-06-09 06:31:50,R_3nC5BcwjMqf2VYr,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,64831b72-1629-bfd9-f3c3-cc7bafd55f5a,2,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3766,2023-07-04 21:13:26,2023-07-04 21:21:15,0,73.235.221.126,100,468,1,2023-07-04 21:21:16,R_3Rg4MmdAXB0aRdH,NaN,...,1,1|2|3,6.295,9.441,9.801,7,64a4dfcd-a0c9-b455-33b4-840ee2266ffa,1,Bing9|Bing11|Human1|Human3|Bing5|Bing4|Human8|...,Q1|Q3|Q6|Q5|Q2|Q4|Q7|Q8|Q222|Q312|Q313|Q314|QI...
3767,2023-07-04 21:46:28,2023-07-04 21:47:48,0,172.58.187.133,100,80,1,2023-07-04 21:47:49,R_3envDB5QAoXagJt,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,64a4e78d-38e9-b42a-2ef7-46fb89672f87,2,NaN,NaN
3768,2023-07-04 22:16:04,2023-07-04 22:16:17,0,98.167.238.41,100,13,1,2023-07-04 22:16:18,R_XGHrW8qzAC9DDZ7,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,64a4ee54-0ba5-ecce-c413-c9ebdf2689b8,2,NaN,NaN
3769,2023-07-04 22:15:02,2023-07-04 22:19:29,0,172.56.222.169,100,266,1,2023-07-04 22:19:30,R_3HiPHhjcqdlOZa6,NaN,...,2,2|1|3,1.716,8.46,8.907,8,64a4ee33-2d93-58fa-a16c-7b23dff07e47,1,Bing3|Human7|Bing10|Human4|Human3|Human10|Huma...,Q1|Q3|Q5|Q2|Q4|Q6|Q7|Q8|Q222|Q312|Q313|Q314|QI...


In [6]:
#filter the compeleted servey
#gc is the embed variable in Qualtrics indicating a completed survey
complete_df = raw_df[raw_df['gc'] == '1'].copy(deep = True)
complete_df

,StartDate,EndDate,Status,IPAddress,Progress,Duration (in seconds),Finished,RecordedDate,ResponseId,RecipientLastName,...,Religious,Religious_DO,Q224_First Click,Q224_Last Click,Q224_Page Submit,Q224_Click Count,rid,gc,FL_12_DO,CRT_DO
30,2023-06-09 06:31:24,2023-06-09 06:37:22,0,65.191.159.34,100,358,1,2023-06-09 06:37:23,R_3MQWR8dWYUcFspR,NaN,...,1,3|1|2,1.888,9.262,9.796,9,64831b7c-d0ac-4a09-d842-e0f483cafce9,1,Bing5|Human10|Bing4|Bing8|Human4|Human7|Bing10...,Q1|Q6|Q5|Q4|Q3|Q2|Q7|Q8|Q222|Q312|Q313|Q314|QI...
32,2023-06-09 06:31:48,2023-06-09 06:37:34,0,73.130.176.48,100,345,1,2023-06-09 06:37:35,R_3Qpw63YR9Sssl5n,NaN,...,1,3|2|1,2.706,12.674,14.497,8,64831b8b-ac7f-5be9-e588-6314fd7b3479,1,Human7|Human2|Bing11|Bing8|Human5|Bing9|Human1...,Q1|Q2|Q3|Q5|Q6|Q4|Q7|Q8|Q222|Q312|Q313|Q314|QI...
45,2023-06-09 06:32:38,2023-06-09 06:39:17,0,69.247.46.67,100,399,1,2023-06-09 06:39:18,R_3nf3WrlmqsG3Ztc,NaN,...,1,3|2|1,4.825,11.901,12.664,6,64831bc4-4ef7-515b-0710-ae58e5d1091f,1,human9|Human3|Human7|Bing8|Bing1|Bing2|Bing7|B...,Q1|Q3|Q4|Q7|Q6|Q5|Q2|Q8|Q222|Q312|Q313|Q314|QI...
55,2023-06-09 06:31:11,2023-06-09 06:39:48,0,68.43.217.238,100,517,1,2023-06-09 06:39:49,R_AFeMJNfCduL9YyJ,NaN,...,1,2|3|1,5.859,13.804,15.865,5,64831b5f-5283-1137-3fb4-169e12615fd2,1,Human4|Bing1|Human2|Bing9|Bing3|Human8|Bing8|B...,Q1|Q3|Q5|Q2|Q4|Q7|Q6|Q8|Q222|Q312|Q313|Q314|QI...
58,2023-06-09 06:33:03,2023-06-09 06:40:04,0,100.35.180.86,100,420,1,2023-06-09 06:40:05,R_yEm3JKR9EvA0zp7,NaN,...,1,1|3|2,1.689,9.509,10.102,7,64831bea-df9c-61cc-efe7-8c25ef953a3b,1,Bing7|Human3|Human1|Bing11|Human4|Bing5|Human8...,Q1|Q7|Q5|Q4|Q3|Q6|Q2|Q8|Q222|Q312|Q313|Q314|QI...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3761,2023-07-04 06:51:22,2023-07-04 07:04:48,0,174.238.97.58,100,805,1,2023-07-04 07:04:49,R_1ikPmfJyq2GSENm,NaN,...,2,2|3|1,3.102,15.336,16.971,6,64a41599-ec04-0617-9431-d26ddfd21bbf,1,Human3|Human8|Human6|Bing8|Bing7|Bing4|Human10...,Q1|Q2|Q5|Q3|Q6|Q7|Q4|Q8|Q222|Q312|Q313|Q314|QI...
3762,2023-07-04 08:10:44,2023-07-04 08:46:55,0,172.58.252.85,100,2170,1,2023-07-04 08:46:56,R_suKzEzU2RMNXYFH,NaN,...,2,2|1|3,4.676,25.216,27.194,8,64a42832-d8b0-b58e-b961-923443552f25,1,Human3|Bing5|Human5|Bing9|Bing10|Human10|Human...,Q1|Q6|Q3|Q4|Q2|Q7|Q5|Q8|Q222|Q312|Q313|Q314|QI...
3763,2023-07-04 16:27:38,2023-07-04 16:35:32,0,74.89.82.244,100,473,1,2023-07-04 16:35:33,R_0C00kf5ULNIwGPv,NaN,...,1,3|2|1,3.271,10.968,15.523,7,64a49cd0-2cde-11c6-655b-f3b03b4c1051,1,Bing9|Human2|Bing4|Human4|Bing2|Bing10|Human7|...,Q1|Q2|Q6|Q5|Q4|Q3|Q7|Q8|Q222|Q312|Q313|Q314|QI...
3766,2023-07-04 21:13:26,2023-07-04 21:21:15,0,73.235.221.126,100,468,1,2023-07-04 21:21:16,R_3Rg4MmdAXB0aRdH,NaN,...,1,1|2|3,6.295,9.441,9.801,7,64a4dfcd-a0c9-b455-33b4-840ee2266ffa,1,Bing9|Bing11|Human1|Human3|Bing5|Bing4|Human8|...,Q1|Q3|Q6|Q5|Q2|Q4|Q7|Q8|Q222|Q312|Q313|Q314|QI...


In [7]:
#Removing the first 50 participants because they were gathered before the pre-registration
complete_df = complete_df[51:]
#comDf.reset_index(inplace=True)
complete_df

,StartDate,EndDate,Status,IPAddress,Progress,Duration (in seconds),Finished,RecordedDate,ResponseId,RecipientLastName,...,Religious,Religious_DO,Q224_First Click,Q224_Last Click,Q224_Page Submit,Q224_Click Count,rid,gc,FL_12_DO,CRT_DO
161,2023-06-22 05:40:46,2023-06-22 05:46:27,0,24.12.59.20,100,340,1,2023-06-22 05:46:28,R_uaX5Dv7sUkY8oFP,NaN,...,1,1|2|3,3.466,9.146,9.793,9,64943320-e8a4-4a07-ced1-9bf079d4983d,1,Bing2|Human3|Bing4|Human7|Bing11|Human10|Bing1...,Q1|Q4|Q6|Q3|Q2|Q7|Q5|Q8|Q222|Q312|Q313|Q314|QI...
165,2023-06-22 05:42:06,2023-06-22 05:50:04,0,98.214.159.93,100,477,1,2023-06-22 05:50:05,R_0N8NtfCM1SRFOtH,NaN,...,2,1|2|3,2.533,12.309,14.464,8,6494337b-ce5e-54f6-fb49-47ec55ed592e,1,Bing5|Human6|human9|Bing4|Bing11|Bing1|Human10...,Q1|Q7|Q3|Q5|Q2|Q6|Q4|Q8|Q222|Q312|Q313|Q314|QI...
166,2023-06-22 05:43:38,2023-06-22 05:50:07,0,173.93.89.212,100,388,1,2023-06-22 05:50:08,R_1pFqGDi1DMW0WYl,NaN,...,2,1|2|3,2.541,9.526,10.163,6,649433e3-dbd1-df3b-534a-3d9036b86165,1,Human1|Bing3|Human5|Human2|Human10|Bing10|Huma...,Q1|Q3|Q5|Q2|Q4|Q7|Q6|Q8|Q222|Q312|Q313|Q314|QI...
169,2023-06-22 05:40:20,2023-06-22 05:52:09,0,172.58.99.9,100,709,1,2023-06-22 05:52:10,R_3g10Deg3oKZO1gd,NaN,...,2,2|3|1,3.551,20.917,22.162,9,6494330d-10d9-c84c-4c2c-d20d44ddab58,1,Human8|Bing10|Human6|human9|Bing2|Bing8|Human7...,Q1|Q2|Q3|Q5|Q7|Q6|Q4|Q8|Q222|Q312|Q313|Q314|QI...
170,2023-06-22 05:43:08,2023-06-22 05:52:15,0,67.10.224.168,100,546,1,2023-06-22 05:52:16,R_2Vlk1Hi2n86oOxd,NaN,...,1,3|2|1,2.639,11.9,13.843,5,649433c0-f368-e08b-7641-b853e4308a5c,1,Bing2|Human8|human9|Bing4|Bing7|Human10|Human6...,Q1|Q4|Q3|Q5|Q7|Q2|Q6|Q8|Q222|Q312|Q313|Q314|QI...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3761,2023-07-04 06:51:22,2023-07-04 07:04:48,0,174.238.97.58,100,805,1,2023-07-04 07:04:49,R_1ikPmfJyq2GSENm,NaN,...,2,2|3|1,3.102,15.336,16.971,6,64a41599-ec04-0617-9431-d26ddfd21bbf,1,Human3|Human8|Human6|Bing8|Bing7|Bing4|Human10...,Q1|Q2|Q5|Q3|Q6|Q7|Q4|Q8|Q222|Q312|Q313|Q314|QI...
3762,2023-07-04 08:10:44,2023-07-04 08:46:55,0,172.58.252.85,100,2170,1,2023-07-04 08:46:56,R_suKzEzU2RMNXYFH,NaN,...,2,2|1|3,4.676,25.216,27.194,8,64a42832-d8b0-b58e-b961-923443552f25,1,Human3|Bing5|Human5|Bing9|Bing10|Human10|Human...,Q1|Q6|Q3|Q4|Q2|Q7|Q5|Q8|Q222|Q312|Q313|Q314|QI...
3763,2023-07-04 16:27:38,2023-07-04 16:35:32,0,74.89.82.244,100,473,1,2023-07-04 16:35:33,R_0C00kf5ULNIwGPv,NaN,...,1,3|2|1,3.271,10.968,15.523,7,64a49cd0-2cde-11c6-655b-f3b03b4c1051,1,Bing9|Human2|Bing4|Human4|Bing2|Bing10|Human7|...,Q1|Q2|Q6|Q5|Q4|Q3|Q7|Q8|Q222|Q312|Q313|Q314|QI...
3766,2023-07-04 21:13:26,2023-07-04 21:21:15,0,73.235.221.126,100,468,1,2023-07-04 21:21:16,R_3Rg4MmdAXB0aRdH,NaN,...,1,1|2|3,6.295,9.441,9.801,7,64a4dfcd-a0c9-b455-33b4-840ee2266ffa,1,Bing9|Bing11|Human1|Human3|Bing5|Bing4|Human8|...,Q1|Q3|Q6|Q5|Q2|Q4|Q7|Q8|Q222|Q312|Q313|Q314|QI...


In [8]:
complete_df_col_name = complete_df.columns.tolist()
complete_df_col_name

['StartDate',
 'EndDate',
 'Status',
 'IPAddress',
 'Progress',
 'Duration (in seconds)',
 'Finished',
 'RecordedDate',
 'ResponseId',
 'RecipientLastName',
 'RecipientFirstName',
 'RecipientEmail',
 'ExternalReference',
 'LocationLatitude',
 'LocationLongitude',
 'DistributionChannel',
 'UserLanguage',
 ' ',
 'Q1_Browser',
 'Q1_Version',
 'Q1_Operating System',
 'Q1_Resolution',
 'Q2',
 'Q2_6_TEXT',
 'Q2_DO',
 'Q3',
 'Q3_6_TEXT',
 'socialMCons ',
 'Q221_First Click',
 'Q221_Last Click',
 'Q221_Page Submit',
 'Q221_Click Count',
 'Attention 1',
 'Attention 1_9_TEXT',
 'Q311_First Click',
 'Q311_Last Click',
 'Q311_Page Submit',
 'Q311_Click Count',
 'Q312_First Click',
 'Q312_Last Click',
 'Q312_Page Submit',
 'Q312_Click Count',
 'Q2.1',
 'Q222_First Click',
 'Q222_Last Click',
 'Q222_Page Submit',
 'Q222_Click Count',
 'Q3.1',
 'Q313_First Click',
 'Q313_Last Click',
 'Q313_Page Submit',
 'Q313_Click Count',
 'Q4',
 'Q314_First Click',
 'Q314_Last Click',
 'Q314_Page Submit',
 'Q314_

In [9]:
#column name transfer list
col_name_transfer = {
    'Statement 1' : 'AI01',
    'Q111_Page Submit' : 'AI01_time',
    'Q237' : 'AI02',
    'Q240_Page Submit' : 'AI02_time',
    'Q241' : 'AI03',
    'Q244_Page Submit' : 'AI03_time',
    'Q245' : 'AI04',
    'Q248_Page Submit' : 'AI04_time',
    'Q253' : 'AI05',
    'Q256_Page Submit' : 'AI05_time',
    'Q261' : 'AI06',
    'Q264_Page Submit' : 'AI06_time',
    'Q265' : 'AI07',
    'Q268_Page Submit' : 'AI07_time',
    'Q269' : 'AI08',
    'Q272_Page Submit' : 'AI08_time',
    'Q273' : 'AI09',
    'Q276_Page Submit' : 'AI09_time',
    'Q277' : 'AI10',
    'Q280_Page Submit' : 'AI10_time',
    'Q281' : 'Hum01',
    'Q284_Page Submit' : 'Hum01_time',
    'Q285' : 'Hum02',
    'Q288_Page Submit' : 'Hum02_time',
    'Q289' : 'Hum03',
    'Q292_Page Submit' : 'Hum03_time',
    'Q293' : 'Hum04',
    'Q296_Page Submit' : 'Hum04_time',
    'Q297' : 'Hum05',
    'Q300_Page Submit' : 'Hum05_time',
    'Q301' : 'Hum06',
    'Q304_Page Submit' : 'Hum06_time',
    'Q305' : 'Hum07',
    'Q308_Page Submit' : 'Hum07_time',
    'Q309' : 'Hum08',
    'Q312_Page Submit.1' : 'Hum08_time',
    'Q313' : 'Hum09',
    'Q316_Page Submit.1' : 'Hum09_time',
    'Q317' : 'Hum10',
    'Q320_Page Submit' : 'Hum10_time',
    'Share 1 ' : 'ShareAI01',
    'Q238' : 'ShareAI02',
    'Q242' : 'ShareAI03',
    'Q246' : 'ShareAI04',
    'Q254' : 'ShareAI05',
    'Q262' : 'ShareAI06',
    'Q266' : 'ShareAI07',
    'Q270' : 'ShareAI08',
    'Q274' : 'ShareAI09',
    'Q278' : 'ShareAI10',
    'Q282' : 'ShareHum01',
    'Q286' : 'ShareHum02',
    'Q290' : 'ShareHum03',
    'Q294' : 'ShareHum04',
    'Q298' : 'ShareHum05',
    'Q302' : 'ShareHum06',
    'Q306' : 'ShareHum07',
    'Q310' : 'ShareHum08',
    'Q314' : 'ShareHum09',
    'Q318' : 'ShareHum10',
    'Q2.1' : 'CRT1',
    'Q3.1' : 'CRT2',
    'Q4' : 'CRT3',
    'Q5' : 'CRT4',
    'Q6' : 'CRT5',
    'Q7' : 'CRT6',
    'Q7.1' : 'PoliticalOrientation',
    'Q9' : 'SocialOrientation',
    'Q10' : 'EconomicOrientation',
    'Q2.2': 'NewsConsumption',
    'Q1_1' : 'Concern',
    'Q1.1' : 'Age',
    'Q2.4' : 'Gender',
    'Q4.1' : 'Income',
    'Q5.1' : 'Ethnicity',
    'Q6.1' : 'Fluency',
    'Q3_1' : 'EducationYears',
    'Religious' : 'Religiosity',
    'FL_12_DO' : 'StatementSequence',
    'Q1' : 'answered_randomly',
    'Q2.3' : 'search_internet',
    'Vaccination' : 'VaccinationStatus',
    'socialMCons ' : 'socialmedia_consumption',
    
}

In [10]:
complete_df = complete_df.rename(columns= col_name_transfer)
complete_df_col_name = complete_df.columns.tolist()
complete_df_col_name


['StartDate',
 'EndDate',
 'Status',
 'IPAddress',
 'Progress',
 'Duration (in seconds)',
 'Finished',
 'RecordedDate',
 'ResponseId',
 'RecipientLastName',
 'RecipientFirstName',
 'RecipientEmail',
 'ExternalReference',
 'LocationLatitude',
 'LocationLongitude',
 'DistributionChannel',
 'UserLanguage',
 ' ',
 'Q1_Browser',
 'Q1_Version',
 'Q1_Operating System',
 'Q1_Resolution',
 'Q2',
 'Q2_6_TEXT',
 'Q2_DO',
 'Q3',
 'Q3_6_TEXT',
 'socialmedia_consumption',
 'Q221_First Click',
 'Q221_Last Click',
 'Q221_Page Submit',
 'Q221_Click Count',
 'Attention 1',
 'Attention 1_9_TEXT',
 'Q311_First Click',
 'Q311_Last Click',
 'Q311_Page Submit',
 'Q311_Click Count',
 'Q312_First Click',
 'Q312_Last Click',
 'Q312_Page Submit',
 'Q312_Click Count',
 'CRT1',
 'Q222_First Click',
 'Q222_Last Click',
 'Q222_Page Submit',
 'Q222_Click Count',
 'CRT2',
 'Q313_First Click',
 'Q313_Last Click',
 'Q313_Page Submit',
 'Q313_Click Count',
 'CRT3',
 'Q314_First Click',
 'Q314_Last Click',
 'Q314_Page Sub

In [11]:
# Remove people who fail the second attention test at the end of the experiment.
complete_df = complete_df.drop(complete_df.loc[(complete_df['Attention 2'] == '3') |(complete_df['Attention 2'] == '4') | (complete_df['Attention 2'] == '5') | (complete_df['Attention 3'] == '1') |(complete_df['Attention 3'] == '2') | (complete_df['Attention 3'] == '3')].index)
complete_df

,StartDate,EndDate,Status,IPAddress,Progress,Duration (in seconds),Finished,RecordedDate,ResponseId,RecipientLastName,...,Religiosity,Religious_DO,Q224_First Click,Q224_Last Click,Q224_Page Submit,Q224_Click Count,rid,gc,StatementSequence,CRT_DO
161,2023-06-22 05:40:46,2023-06-22 05:46:27,0,24.12.59.20,100,340,1,2023-06-22 05:46:28,R_uaX5Dv7sUkY8oFP,NaN,...,1,1|2|3,3.466,9.146,9.793,9,64943320-e8a4-4a07-ced1-9bf079d4983d,1,Bing2|Human3|Bing4|Human7|Bing11|Human10|Bing1...,Q1|Q4|Q6|Q3|Q2|Q7|Q5|Q8|Q222|Q312|Q313|Q314|QI...
165,2023-06-22 05:42:06,2023-06-22 05:50:04,0,98.214.159.93,100,477,1,2023-06-22 05:50:05,R_0N8NtfCM1SRFOtH,NaN,...,2,1|2|3,2.533,12.309,14.464,8,6494337b-ce5e-54f6-fb49-47ec55ed592e,1,Bing5|Human6|human9|Bing4|Bing11|Bing1|Human10...,Q1|Q7|Q3|Q5|Q2|Q6|Q4|Q8|Q222|Q312|Q313|Q314|QI...
166,2023-06-22 05:43:38,2023-06-22 05:50:07,0,173.93.89.212,100,388,1,2023-06-22 05:50:08,R_1pFqGDi1DMW0WYl,NaN,...,2,1|2|3,2.541,9.526,10.163,6,649433e3-dbd1-df3b-534a-3d9036b86165,1,Human1|Bing3|Human5|Human2|Human10|Bing10|Huma...,Q1|Q3|Q5|Q2|Q4|Q7|Q6|Q8|Q222|Q312|Q313|Q314|QI...
169,2023-06-22 05:40:20,2023-06-22 05:52:09,0,172.58.99.9,100,709,1,2023-06-22 05:52:10,R_3g10Deg3oKZO1gd,NaN,...,2,2|3|1,3.551,20.917,22.162,9,6494330d-10d9-c84c-4c2c-d20d44ddab58,1,Human8|Bing10|Human6|human9|Bing2|Bing8|Human7...,Q1|Q2|Q3|Q5|Q7|Q6|Q4|Q8|Q222|Q312|Q313|Q314|QI...
170,2023-06-22 05:43:08,2023-06-22 05:52:15,0,67.10.224.168,100,546,1,2023-06-22 05:52:16,R_2Vlk1Hi2n86oOxd,NaN,...,1,3|2|1,2.639,11.9,13.843,5,649433c0-f368-e08b-7641-b853e4308a5c,1,Bing2|Human8|human9|Bing4|Bing7|Human10|Human6...,Q1|Q4|Q3|Q5|Q7|Q2|Q6|Q8|Q222|Q312|Q313|Q314|QI...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3749,2023-07-03 16:49:55,2023-07-03 17:02:55,0,99.172.120.102,100,780,1,2023-07-03 17:02:56,R_0UIkZWsVxsDsNwJ,NaN,...,1,2|1|3,2.907,13.563,15.537,5,64a3508a-fbb1-b99b-3a7e-62a775f8e942,1,Human2|Human6|Bing11|Bing4|Bing5|Human3|Human1...,Q1|Q3|Q7|Q6|Q4|Q2|Q5|Q8|Q222|Q312|Q313|Q314|QI...
3761,2023-07-04 06:51:22,2023-07-04 07:04:48,0,174.238.97.58,100,805,1,2023-07-04 07:04:49,R_1ikPmfJyq2GSENm,NaN,...,2,2|3|1,3.102,15.336,16.971,6,64a41599-ec04-0617-9431-d26ddfd21bbf,1,Human3|Human8|Human6|Bing8|Bing7|Bing4|Human10...,Q1|Q2|Q5|Q3|Q6|Q7|Q4|Q8|Q222|Q312|Q313|Q314|QI...
3762,2023-07-04 08:10:44,2023-07-04 08:46:55,0,172.58.252.85,100,2170,1,2023-07-04 08:46:56,R_suKzEzU2RMNXYFH,NaN,...,2,2|1|3,4.676,25.216,27.194,8,64a42832-d8b0-b58e-b961-923443552f25,1,Human3|Bing5|Human5|Bing9|Bing10|Human10|Human...,Q1|Q6|Q3|Q4|Q2|Q7|Q5|Q8|Q222|Q312|Q313|Q314|QI...
3763,2023-07-04 16:27:38,2023-07-04 16:35:32,0,74.89.82.244,100,473,1,2023-07-04 16:35:33,R_0C00kf5ULNIwGPv,NaN,...,1,3|2|1,3.271,10.968,15.523,7,64a49cd0-2cde-11c6-655b-f3b03b4c1051,1,Bing9|Human2|Bing4|Human4|Bing2|Bing10|Human7|...,Q1|Q2|Q6|Q5|Q4|Q3|Q7|Q8|Q222|Q312|Q313|Q314|QI...


In [12]:
# Removing participants who mentioned they answered randomly or searched internet
complete_df = complete_df.drop(complete_df.loc[(complete_df['answered_randomly'] == '1') | (complete_df['search_internet'] == '1')].index)


complete_df

,StartDate,EndDate,Status,IPAddress,Progress,Duration (in seconds),Finished,RecordedDate,ResponseId,RecipientLastName,...,Religiosity,Religious_DO,Q224_First Click,Q224_Last Click,Q224_Page Submit,Q224_Click Count,rid,gc,StatementSequence,CRT_DO
165,2023-06-22 05:42:06,2023-06-22 05:50:04,0,98.214.159.93,100,477,1,2023-06-22 05:50:05,R_0N8NtfCM1SRFOtH,NaN,...,2,1|2|3,2.533,12.309,14.464,8,6494337b-ce5e-54f6-fb49-47ec55ed592e,1,Bing5|Human6|human9|Bing4|Bing11|Bing1|Human10...,Q1|Q7|Q3|Q5|Q2|Q6|Q4|Q8|Q222|Q312|Q313|Q314|QI...
166,2023-06-22 05:43:38,2023-06-22 05:50:07,0,173.93.89.212,100,388,1,2023-06-22 05:50:08,R_1pFqGDi1DMW0WYl,NaN,...,2,1|2|3,2.541,9.526,10.163,6,649433e3-dbd1-df3b-534a-3d9036b86165,1,Human1|Bing3|Human5|Human2|Human10|Bing10|Huma...,Q1|Q3|Q5|Q2|Q4|Q7|Q6|Q8|Q222|Q312|Q313|Q314|QI...
169,2023-06-22 05:40:20,2023-06-22 05:52:09,0,172.58.99.9,100,709,1,2023-06-22 05:52:10,R_3g10Deg3oKZO1gd,NaN,...,2,2|3|1,3.551,20.917,22.162,9,6494330d-10d9-c84c-4c2c-d20d44ddab58,1,Human8|Bing10|Human6|human9|Bing2|Bing8|Human7...,Q1|Q2|Q3|Q5|Q7|Q6|Q4|Q8|Q222|Q312|Q313|Q314|QI...
170,2023-06-22 05:43:08,2023-06-22 05:52:15,0,67.10.224.168,100,546,1,2023-06-22 05:52:16,R_2Vlk1Hi2n86oOxd,NaN,...,1,3|2|1,2.639,11.9,13.843,5,649433c0-f368-e08b-7641-b853e4308a5c,1,Bing2|Human8|human9|Bing4|Bing7|Human10|Human6...,Q1|Q4|Q3|Q5|Q7|Q2|Q6|Q8|Q222|Q312|Q313|Q314|QI...
173,2023-06-22 05:42:07,2023-06-22 05:52:59,0,76.24.201.1,100,652,1,2023-06-22 05:53:00,R_3fMDXCa61fcNdks,NaN,...,1,1|3|2,15.23,26.016,28.231,4,64943387-ef89-adb4-347b-d5bae990e90d,1,Bing7|Bing8|human9|Human7|Human10|Human4|Bing4...,Q1|Q6|Q2|Q3|Q5|Q4|Q7|Q8|Q222|Q312|Q313|Q314|QI...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3748,2023-07-03 15:41:34,2023-07-03 15:53:29,0,72.24.229.163,100,715,1,2023-07-03 15:53:31,R_2TLhKEQvhBtVeeF,NaN,...,1,2|3|1,2.617,10.945,12.442,4,64a34084-7479-2f95-084c-b175d4da4876,1,Bing4|Bing7|Bing8|Human5|Human6|Bing5|Human3|H...,Q1|Q2|Q5|Q6|Q3|Q7|Q4|Q8|Q222|Q312|Q313|Q314|QI...
3749,2023-07-03 16:49:55,2023-07-03 17:02:55,0,99.172.120.102,100,780,1,2023-07-03 17:02:56,R_0UIkZWsVxsDsNwJ,NaN,...,1,2|1|3,2.907,13.563,15.537,5,64a3508a-fbb1-b99b-3a7e-62a775f8e942,1,Human2|Human6|Bing11|Bing4|Bing5|Human3|Human1...,Q1|Q3|Q7|Q6|Q4|Q2|Q5|Q8|Q222|Q312|Q313|Q314|QI...
3761,2023-07-04 06:51:22,2023-07-04 07:04:48,0,174.238.97.58,100,805,1,2023-07-04 07:04:49,R_1ikPmfJyq2GSENm,NaN,...,2,2|3|1,3.102,15.336,16.971,6,64a41599-ec04-0617-9431-d26ddfd21bbf,1,Human3|Human8|Human6|Bing8|Bing7|Bing4|Human10...,Q1|Q2|Q5|Q3|Q6|Q7|Q4|Q8|Q222|Q312|Q313|Q314|QI...
3762,2023-07-04 08:10:44,2023-07-04 08:46:55,0,172.58.252.85,100,2170,1,2023-07-04 08:46:56,R_suKzEzU2RMNXYFH,NaN,...,2,2|1|3,4.676,25.216,27.194,8,64a42832-d8b0-b58e-b961-923443552f25,1,Human3|Bing5|Human5|Bing9|Bing10|Human10|Human...,Q1|Q6|Q3|Q4|Q2|Q7|Q5|Q8|Q222|Q312|Q313|Q314|QI...


In [13]:
# Remove participants with age less than 10
complete_df['Age'] = complete_df['Age'].astype(int)
complete_df = complete_df.drop(complete_df.loc[(complete_df['Age'] < 18)].index)
complete_df

,StartDate,EndDate,Status,IPAddress,Progress,Duration (in seconds),Finished,RecordedDate,ResponseId,RecipientLastName,...,Religiosity,Religious_DO,Q224_First Click,Q224_Last Click,Q224_Page Submit,Q224_Click Count,rid,gc,StatementSequence,CRT_DO
165,2023-06-22 05:42:06,2023-06-22 05:50:04,0,98.214.159.93,100,477,1,2023-06-22 05:50:05,R_0N8NtfCM1SRFOtH,NaN,...,2,1|2|3,2.533,12.309,14.464,8,6494337b-ce5e-54f6-fb49-47ec55ed592e,1,Bing5|Human6|human9|Bing4|Bing11|Bing1|Human10...,Q1|Q7|Q3|Q5|Q2|Q6|Q4|Q8|Q222|Q312|Q313|Q314|QI...
166,2023-06-22 05:43:38,2023-06-22 05:50:07,0,173.93.89.212,100,388,1,2023-06-22 05:50:08,R_1pFqGDi1DMW0WYl,NaN,...,2,1|2|3,2.541,9.526,10.163,6,649433e3-dbd1-df3b-534a-3d9036b86165,1,Human1|Bing3|Human5|Human2|Human10|Bing10|Huma...,Q1|Q3|Q5|Q2|Q4|Q7|Q6|Q8|Q222|Q312|Q313|Q314|QI...
169,2023-06-22 05:40:20,2023-06-22 05:52:09,0,172.58.99.9,100,709,1,2023-06-22 05:52:10,R_3g10Deg3oKZO1gd,NaN,...,2,2|3|1,3.551,20.917,22.162,9,6494330d-10d9-c84c-4c2c-d20d44ddab58,1,Human8|Bing10|Human6|human9|Bing2|Bing8|Human7...,Q1|Q2|Q3|Q5|Q7|Q6|Q4|Q8|Q222|Q312|Q313|Q314|QI...
170,2023-06-22 05:43:08,2023-06-22 05:52:15,0,67.10.224.168,100,546,1,2023-06-22 05:52:16,R_2Vlk1Hi2n86oOxd,NaN,...,1,3|2|1,2.639,11.9,13.843,5,649433c0-f368-e08b-7641-b853e4308a5c,1,Bing2|Human8|human9|Bing4|Bing7|Human10|Human6...,Q1|Q4|Q3|Q5|Q7|Q2|Q6|Q8|Q222|Q312|Q313|Q314|QI...
173,2023-06-22 05:42:07,2023-06-22 05:52:59,0,76.24.201.1,100,652,1,2023-06-22 05:53:00,R_3fMDXCa61fcNdks,NaN,...,1,1|3|2,15.23,26.016,28.231,4,64943387-ef89-adb4-347b-d5bae990e90d,1,Bing7|Bing8|human9|Human7|Human10|Human4|Bing4...,Q1|Q6|Q2|Q3|Q5|Q4|Q7|Q8|Q222|Q312|Q313|Q314|QI...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3748,2023-07-03 15:41:34,2023-07-03 15:53:29,0,72.24.229.163,100,715,1,2023-07-03 15:53:31,R_2TLhKEQvhBtVeeF,NaN,...,1,2|3|1,2.617,10.945,12.442,4,64a34084-7479-2f95-084c-b175d4da4876,1,Bing4|Bing7|Bing8|Human5|Human6|Bing5|Human3|H...,Q1|Q2|Q5|Q6|Q3|Q7|Q4|Q8|Q222|Q312|Q313|Q314|QI...
3749,2023-07-03 16:49:55,2023-07-03 17:02:55,0,99.172.120.102,100,780,1,2023-07-03 17:02:56,R_0UIkZWsVxsDsNwJ,NaN,...,1,2|1|3,2.907,13.563,15.537,5,64a3508a-fbb1-b99b-3a7e-62a775f8e942,1,Human2|Human6|Bing11|Bing4|Bing5|Human3|Human1...,Q1|Q3|Q7|Q6|Q4|Q2|Q5|Q8|Q222|Q312|Q313|Q314|QI...
3761,2023-07-04 06:51:22,2023-07-04 07:04:48,0,174.238.97.58,100,805,1,2023-07-04 07:04:49,R_1ikPmfJyq2GSENm,NaN,...,2,2|3|1,3.102,15.336,16.971,6,64a41599-ec04-0617-9431-d26ddfd21bbf,1,Human3|Human8|Human6|Bing8|Bing7|Bing4|Human10...,Q1|Q2|Q5|Q3|Q6|Q7|Q4|Q8|Q222|Q312|Q313|Q314|QI...
3762,2023-07-04 08:10:44,2023-07-04 08:46:55,0,172.58.252.85,100,2170,1,2023-07-04 08:46:56,R_suKzEzU2RMNXYFH,NaN,...,2,2|1|3,4.676,25.216,27.194,8,64a42832-d8b0-b58e-b961-923443552f25,1,Human3|Bing5|Human5|Bing9|Bing10|Human10|Human...,Q1|Q6|Q3|Q4|Q2|Q7|Q5|Q8|Q222|Q312|Q313|Q314|QI...


In [14]:
complete_df['Fluency'].value_counts()

1    976
2     12
Name: Fluency, dtype: int64

# Data Correction

In [15]:
# Decode numeric numbers to corresponding values in the survey
# Create gender_binary column which codes male to 0, female to 1 and the other to -1

complete_df['GenderBinary'] = -1
complete_df.loc[complete_df['Gender'] == '1', 'GenderBinary'] = 0
complete_df.loc[complete_df['Gender'] == '2', 'GenderBinary'] = 1

complete_df= complete_df.replace({'Gender' : {'1':'Male','2':'Female','3' : 'Transgender Female' \
                                              , '4':'Transgender Male','5':'Trans/non binary' \
                                              ,'6':'Not listed','7':'Prefer_not_to_answer'}})


In [16]:
complete_df= complete_df.replace({'Fluency' : {'2':'0'}})


In [17]:
#check the inputs for the last CRT test as they should have written a word. This is neceassary to account for mistakes in typing

complete_df['CRT6'].unique()

array(['Emily', 'June', 'june', 'June ', 'emily', 'Emily ', 'May',
       'What ', '0', "Don't know ", 'emily’s ', 'jun', 'JUNE', 'june ',
       'Unknown', 'junio', 'emily ', 'unknown', 'Emiily', 'Na', 'may',
       'Jun', "I don't know", 'Jume', 'Sandra', "I don't fucking know ",
       'EMILY', 'Emiy', '8', 'march', 'Don’t know but maybe June', '5'],
      dtype=object)

In [18]:
# Modify misspelled correct answers
complete_df['CRT6'] = complete_df['CRT6'].str.strip()
complete_df['CRT6'] = complete_df['CRT6'].str.strip("\'s")
complete_df['CRT6'] = complete_df['CRT6'].str.strip('\'s')
complete_df['CRT6'] = complete_df['CRT6'].str.strip('\'')
complete_df['CRT6'] = complete_df['CRT6'].str.replace('Emiily','emily')
complete_df['CRT6'] = complete_df['CRT6'].str.replace('Emily’','emily')
complete_df['CRT6'] = complete_df['CRT6'].str.replace('emily’','emily')
complete_df['CRT6'] = complete_df['CRT6'].str.replace('EMILY','emily')
complete_df['CRT6'] = complete_df['CRT6'].str.replace('Emlly','emily')
complete_df['CRT6'] = complete_df['CRT6'].str.replace('Emiy','emily')
complete_df['CRT6'].unique()

array(['Emily', 'June', 'june', 'emily', 'May', 'What', '0', "Don't know",
       'jun', 'JUNE', 'Unknown', 'junio', 'unknown', 'Na', 'may', 'Jun',
       "I don't know", 'Jume', 'Sandra', "I don't fucking know", '8',
       'march', 'Don’t know but maybe June', '5'], dtype=object)

In [19]:
# Calculating CRT scores. Each correct answer is 1 point so the range is 0 to 6
crt_answer = complete_df[['CRT1','CRT2','CRT3','CRT4','CRT5','CRT6','ResponseId']].copy(deep = True)
crt_answer.reset_index(inplace = True, drop = True)
solution_crt = pd.DataFrame({
    'answer': ['4','10','39','2','8','emily']}, index = ['CRT1','CRT2','CRT3','CRT4','CRT5','CRT6'])

crt_answer['CRT'] = ((solution_crt['answer'][0] == crt_answer['CRT1']).astype(int) \
                           + (solution_crt['answer'][1] == crt_answer['CRT2']).astype(int) \
                           + (solution_crt['answer'][2] == crt_answer['CRT3']).astype(int) \
                           + (solution_crt['answer'][3] == crt_answer['CRT4']).astype(int) 
                           + (solution_crt['answer'][4] == crt_answer['CRT5']).astype(int) \
                           + (solution_crt['answer'][5] == crt_answer['CRT6'].str.lower()).astype(int))
crt_answer

,CRT1,CRT2,CRT3,CRT4,CRT5,CRT6,ResponseId,CRT
0,8,5,20,1,8,Emily,R_0N8NtfCM1SRFOtH,2
1,8,5,10,2,8,Emily,R_1pFqGDi1DMW0WYl,3
2,8,50,20,1,7,Emily,R_3g10Deg3oKZO1gd,1
3,4,10,39,1,7,June,R_2Vlk1Hi2n86oOxd,3
4,8,50,20,1,7,june,R_3fMDXCa61fcNdks,0
...,...,...,...,...,...,...,...,...
983,8,50,20,2,7,Emily,R_2TLhKEQvhBtVeeF,2
984,8,50,20,1,7,june,R_0UIkZWsVxsDsNwJ,0
985,8,50,39,1,8,Emily,R_1ikPmfJyq2GSENm,3
986,8,100,5,1,7,June,R_suKzEzU2RMNXYFH,0


In [20]:
complete_df = complete_df.merge(crt_answer[['CRT','ResponseId']], on = 'ResponseId')
complete_df

,StartDate,EndDate,Status,IPAddress,Progress,Duration (in seconds),Finished,RecordedDate,ResponseId,RecipientLastName,...,Q224_First Click,Q224_Last Click,Q224_Page Submit,Q224_Click Count,rid,gc,StatementSequence,CRT_DO,GenderBinary,CRT
0,2023-06-22 05:42:06,2023-06-22 05:50:04,0,98.214.159.93,100,477,1,2023-06-22 05:50:05,R_0N8NtfCM1SRFOtH,NaN,...,2.533,12.309,14.464,8,6494337b-ce5e-54f6-fb49-47ec55ed592e,1,Bing5|Human6|human9|Bing4|Bing11|Bing1|Human10...,Q1|Q7|Q3|Q5|Q2|Q6|Q4|Q8|Q222|Q312|Q313|Q314|QI...,1,2
1,2023-06-22 05:43:38,2023-06-22 05:50:07,0,173.93.89.212,100,388,1,2023-06-22 05:50:08,R_1pFqGDi1DMW0WYl,NaN,...,2.541,9.526,10.163,6,649433e3-dbd1-df3b-534a-3d9036b86165,1,Human1|Bing3|Human5|Human2|Human10|Bing10|Huma...,Q1|Q3|Q5|Q2|Q4|Q7|Q6|Q8|Q222|Q312|Q313|Q314|QI...,1,3
2,2023-06-22 05:40:20,2023-06-22 05:52:09,0,172.58.99.9,100,709,1,2023-06-22 05:52:10,R_3g10Deg3oKZO1gd,NaN,...,3.551,20.917,22.162,9,6494330d-10d9-c84c-4c2c-d20d44ddab58,1,Human8|Bing10|Human6|human9|Bing2|Bing8|Human7...,Q1|Q2|Q3|Q5|Q7|Q6|Q4|Q8|Q222|Q312|Q313|Q314|QI...,1,1
3,2023-06-22 05:43:08,2023-06-22 05:52:15,0,67.10.224.168,100,546,1,2023-06-22 05:52:16,R_2Vlk1Hi2n86oOxd,NaN,...,2.639,11.9,13.843,5,649433c0-f368-e08b-7641-b853e4308a5c,1,Bing2|Human8|human9|Bing4|Bing7|Human10|Human6...,Q1|Q4|Q3|Q5|Q7|Q2|Q6|Q8|Q222|Q312|Q313|Q314|QI...,1,3
4,2023-06-22 05:42:07,2023-06-22 05:52:59,0,76.24.201.1,100,652,1,2023-06-22 05:53:00,R_3fMDXCa61fcNdks,NaN,...,15.23,26.016,28.231,4,64943387-ef89-adb4-347b-d5bae990e90d,1,Bing7|Bing8|human9|Human7|Human10|Human4|Bing4...,Q1|Q6|Q2|Q3|Q5|Q4|Q7|Q8|Q222|Q312|Q313|Q314|QI...,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
983,2023-07-03 15:41:34,2023-07-03 15:53:29,0,72.24.229.163,100,715,1,2023-07-03 15:53:31,R_2TLhKEQvhBtVeeF,NaN,...,2.617,10.945,12.442,4,64a34084-7479-2f95-084c-b175d4da4876,1,Bing4|Bing7|Bing8|Human5|Human6|Bing5|Human3|H...,Q1|Q2|Q5|Q6|Q3|Q7|Q4|Q8|Q222|Q312|Q313|Q314|QI...,1,2
984,2023-07-03 16:49:55,2023-07-03 17:02:55,0,99.172.120.102,100,780,1,2023-07-03 17:02:56,R_0UIkZWsVxsDsNwJ,NaN,...,2.907,13.563,15.537,5,64a3508a-fbb1-b99b-3a7e-62a775f8e942,1,Human2|Human6|Bing11|Bing4|Bing5|Human3|Human1...,Q1|Q3|Q7|Q6|Q4|Q2|Q5|Q8|Q222|Q312|Q313|Q314|QI...,0,0
985,2023-07-04 06:51:22,2023-07-04 07:04:48,0,174.238.97.58,100,805,1,2023-07-04 07:04:49,R_1ikPmfJyq2GSENm,NaN,...,3.102,15.336,16.971,6,64a41599-ec04-0617-9431-d26ddfd21bbf,1,Human3|Human8|Human6|Bing8|Bing7|Bing4|Human10...,Q1|Q2|Q5|Q3|Q6|Q7|Q4|Q8|Q222|Q312|Q313|Q314|QI...,1,3
986,2023-07-04 08:10:44,2023-07-04 08:46:55,0,172.58.252.85,100,2170,1,2023-07-04 08:46:56,R_suKzEzU2RMNXYFH,NaN,...,4.676,25.216,27.194,8,64a42832-d8b0-b58e-b961-923443552f25,1,Human3|Bing5|Human5|Bing9|Bing10|Human10|Human...,Q1|Q6|Q3|Q4|Q2|Q7|Q5|Q8|Q222|Q312|Q313|Q314|QI...,1,0


In [21]:
# Adding seperator to the start and the end for purpose of counting
complete_df['StatementSequence'] = '|' + complete_df['StatementSequence'] + '|'

In [22]:
# Converting multiple ethnicity choives to other choice
complete_df.loc[complete_df['Ethnicity'].str.contains(','), 'Ethnicity'] = '8'

# Create new binary varibale for ethnicity 1 for white and 0 for the rest
complete_df.loc[complete_df['Ethnicity'].str.contains('1'), 'EthnicityBinary'] = 1
complete_df['EthnicityBinary'].fillna(0, inplace=True)

In [23]:
# Converting income level to categories: below 30K$: low (code 0), 
# 30K$-80K$: middle (code 1) , above 80K$ high(code 2)

complete_df['Income'] = complete_df['Income'].astype(int)
income_bins = [-float('inf'), 3, 8, float('inf')]
income_labels = [0, 1, 2]

# Categorize the income
complete_df['Income'] = pd.cut(complete_df['Income'], bins=income_bins, labels=income_labels).astype(int)


# Creaeting dataframe for regression

In [24]:
unique_participant = complete_df['ResponseId'].unique()
STATEMENT_LIST_LENGTH = len(unique_participant)

In [25]:
statement_list = ['Hum01'] * STATEMENT_LIST_LENGTH \
                 + ['Hum02'] * STATEMENT_LIST_LENGTH \
                 + ['Hum03'] * STATEMENT_LIST_LENGTH \
                 + ['Hum04'] * STATEMENT_LIST_LENGTH \
                 + ['Hum05'] * STATEMENT_LIST_LENGTH \
                 + ['Hum06'] * STATEMENT_LIST_LENGTH \
                 + ['Hum07'] * STATEMENT_LIST_LENGTH \
                 + ['Hum08'] * STATEMENT_LIST_LENGTH \
                 + ['Hum09'] * STATEMENT_LIST_LENGTH \
                 + ['Hum10'] * STATEMENT_LIST_LENGTH \
                 + ['AI01'] * STATEMENT_LIST_LENGTH \
                 + ['AI02'] * STATEMENT_LIST_LENGTH \
                 + ['AI03'] * STATEMENT_LIST_LENGTH \
                 + ['AI04'] * STATEMENT_LIST_LENGTH \
                 + ['AI05'] * STATEMENT_LIST_LENGTH \
                 + ['AI06'] * STATEMENT_LIST_LENGTH \
                 + ['AI07'] * STATEMENT_LIST_LENGTH \
                 + ['AI08'] * STATEMENT_LIST_LENGTH \
                 + ['AI09'] * STATEMENT_LIST_LENGTH \
                 + ['AI10'] * STATEMENT_LIST_LENGTH

In [26]:
fake_news_generation_source = np.zeros(STATEMENT_LIST_LENGTH * NUM_HUMAN_STATEMENTS, dtype=int).tolist() \
              + np.ones(STATEMENT_LIST_LENGTH * NUM_AI_STATEMENTS, dtype=int).tolist()

In [27]:
regression_df_accuracy = pd.DataFrame(columns = ['ResponseId','Statement','fake_news_generation_source','PerceivedVeracity','Time_to_respond'])
regression_df_accuracy['PerceivedVeracity'] = pd.concat([complete_df['Hum01'], complete_df['Hum02'], complete_df['Hum03']
                                                , complete_df['Hum04'], complete_df['Hum05'], complete_df['Hum06']
                                                , complete_df['Hum07'], complete_df['Hum08'],complete_df['Hum09']
                                                , complete_df['Hum10'],complete_df['AI01'], complete_df['AI02']
                                                , complete_df['AI03'], complete_df['AI04'], complete_df['AI05']
                                                , complete_df['AI06'], complete_df['AI07'], complete_df['AI08']
                                                , complete_df['AI09'],complete_df['AI10']], ignore_index=True)
regression_df_accuracy['WillingnessToShare'] = pd.concat([complete_df['ShareHum01'], complete_df['ShareHum02']
                                                   , complete_df['ShareHum03'], complete_df['ShareHum04'] 
                                                   , complete_df['ShareHum05'], complete_df['ShareHum06'] 
                                                   , complete_df['ShareHum07'], complete_df['ShareHum08']
                                                   , complete_df['ShareHum09'],complete_df['ShareHum10']
                                                   , complete_df['ShareAI01'], complete_df['ShareAI02'] 
                                                   , complete_df['ShareAI03'], complete_df['ShareAI04'] 
                                                   , complete_df['ShareAI05'], complete_df['ShareAI06'] 
                                                   , complete_df['ShareAI07'], complete_df['ShareAI08']
                                                   , complete_df['ShareAI09'],complete_df['ShareAI10']]
                                                  , ignore_index=True)
regression_df_accuracy['Time_to_respond'] = pd.concat([complete_df['Hum01_time'], complete_df['Hum02_time']
                                                       , complete_df['Hum03_time'], complete_df['Hum04_time']
                                                       , complete_df['Hum05_time'], complete_df['Hum06_time']
                                                       , complete_df['Hum07_time'], complete_df['Hum08_time']
                                                       , complete_df['Hum09_time'], complete_df['Hum10_time']
                                                       , complete_df['AI01_time'], complete_df['AI02_time']
                                                       , complete_df['AI03_time'], complete_df['AI04_time']
                                                       , complete_df['AI05_time'], complete_df['AI06_time']
                                                       , complete_df['AI07_time'], complete_df['AI08_time']
                                                       , complete_df['AI09_time'],complete_df['AI10_time']]
                                                       , ignore_index=True)
regression_df_accuracy['ResponseId'] = pd.concat([complete_df['ResponseId']] * 20, ignore_index=True)
regression_df_accuracy['Statement'] = pd.concat([pd.DataFrame(statement_list)], ignore_index=True)
regression_df_accuracy['fake_news_generation_source'] = pd.concat([pd.DataFrame(fake_news_generation_source)], ignore_index=True)


In [28]:
regression_df_accuracy = regression_df_accuracy.merge(complete_df[['ResponseId', 'VaccinationStatus', 'SocialOrientation',
                                                            'EconomicOrientation', 'PoliticalOrientation', 'Concern', 'NewsConsumption',
                                                            'CRT', 'Age', 'Gender', 'Religiosity', 'Income', 'Fluency',
                                                            'EducationYears', 'Ethnicity', 'StatementSequence', 'EthnicityBinary','GenderBinary']], on = 'ResponseId')
regression_df_accuracy[['PerceivedVeracity', 'WillingnessToShare', 'VaccinationStatus', 'SocialOrientation',
                                                                        'EconomicOrientation', 'PoliticalOrientation', 'Religiosity',
                                                                        'EducationYears', 'Ethnicity', 'EthnicityBinary','GenderBinary',
                                                                        'Concern','NewsConsumption','Fluency']] = regression_df_accuracy[['PerceivedVeracity',
                                                                        'WillingnessToShare', 'VaccinationStatus', 'SocialOrientation',
                                                                        'EconomicOrientation', 'PoliticalOrientation', 'Religiosity',
                                                                        'EducationYears', 'Ethnicity', 'EthnicityBinary','GenderBinary',
                                                                        'Concern','NewsConsumption','Fluency']].astype(int)
regression_df_accuracy['Time_to_respond'] = regression_df_accuracy['Time_to_respond'].astype(float)
regression_df_accuracy[['ResponseId', 'Statement']] = regression_df_accuracy[['ResponseId', 'Statement']].astype(str)
regression_df_accuracy['VaccinationStatus'] = regression_df_accuracy['VaccinationStatus'].replace(2, 0)
regression_df_accuracy['Time_to_respond'] = regression_df_accuracy['Time_to_respond']/ 2
regression_df_accuracy['WillingnessToShare'] = regression_df_accuracy['WillingnessToShare'].replace(1, 0)  
regression_df_accuracy['WillingnessToShare'] = regression_df_accuracy['WillingnessToShare'].replace(3, 1)
regression_df_accuracy['WillingnessToShare'] = regression_df_accuracy['WillingnessToShare'].replace(2, -1)
regression_df_accuracy

,ResponseId,Statement,fake_news_generation_source,PerceivedVeracity,Time_to_respond,WillingnessToShare,VaccinationStatus,SocialOrientation,EconomicOrientation,PoliticalOrientation,...,Age,Gender,Religiosity,Income,Fluency,EducationYears,Ethnicity,StatementSequence,EthnicityBinary,GenderBinary
0,R_0N8NtfCM1SRFOtH,Hum01,0,1,3.3420,0,1,1,1,1,...,58,Female,2,1,1,15,1,|Bing5|Human6|human9|Bing4|Bing11|Bing1|Human1...,1,1
1,R_0N8NtfCM1SRFOtH,Hum02,0,3,6.5025,0,1,1,1,1,...,58,Female,2,1,1,15,1,|Bing5|Human6|human9|Bing4|Bing11|Bing1|Human1...,1,1
2,R_0N8NtfCM1SRFOtH,Hum03,0,1,2.7935,0,1,1,1,1,...,58,Female,2,1,1,15,1,|Bing5|Human6|human9|Bing4|Bing11|Bing1|Human1...,1,1
3,R_0N8NtfCM1SRFOtH,Hum04,0,2,6.1750,0,1,1,1,1,...,58,Female,2,1,1,15,1,|Bing5|Human6|human9|Bing4|Bing11|Bing1|Human1...,1,1
4,R_0N8NtfCM1SRFOtH,Hum05,0,1,2.6790,0,1,1,1,1,...,58,Female,2,1,1,15,1,|Bing5|Human6|human9|Bing4|Bing11|Bing1|Human1...,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19755,R_0C00kf5ULNIwGPv,AI06,1,1,2.9300,0,1,5,5,3,...,67,Female,1,1,1,20,1,|Bing9|Human2|Bing4|Human4|Bing2|Bing10|Human7...,1,1
19756,R_0C00kf5ULNIwGPv,AI07,1,2,4.0460,0,1,5,5,3,...,67,Female,1,1,1,20,1,|Bing9|Human2|Bing4|Human4|Bing2|Bing10|Human7...,1,1
19757,R_0C00kf5ULNIwGPv,AI08,1,2,9.8465,0,1,5,5,3,...,67,Female,1,1,1,20,1,|Bing9|Human2|Bing4|Human4|Bing2|Bing10|Human7...,1,1
19758,R_0C00kf5ULNIwGPv,AI09,1,1,5.0180,0,1,5,5,3,...,67,Female,1,1,1,20,1,|Bing9|Human2|Bing4|Human4|Bing2|Bing10|Human7...,1,1


In [29]:
# This is a binary variable which is 1 for people choose stongly agree and somehow agree and 0 otherwise
regression_df_accuracy['fall_for_fake_news'] = regression_df_accuracy['PerceivedVeracity'].apply(lambda x: 0 if (x == 0 or x == 1 or x== 2) else 1)

In [30]:
regression_df_accuracy['StatementSequence'] = regression_df_accuracy['StatementSequence'].apply(lambda x: renamingSequenceString(x))
regression_df_accuracy['StatementSequence'] = regression_df_accuracy.apply(lambda x: findStatementSequence(x['StatementSequence'] , x['Statement']),axis = 1)

In [31]:
# Coding the source of human-generated fake news source: 0 is from snopes.com, 1 from politifact.com, 2 from Pennycook's article fighithng covid 2020
# For the sake of consitency we code the AI-generated fake news as 3 from Bing chat

regression_df_accuracy['statement_source'] = regression_df_accuracy['Statement'].apply(
    lambda x: 0 if x in ["Hum01", "Hum02", "Hum03", "Hum04"] else 1 if x in ["Hum05", "Hum06"] else 2 if x in ["Hum07", "Hum08", "Hum09", "Hum10"]  else 3
)

In [32]:
cols_to_update = ['PerceivedVeracity', 'SocialOrientation', 'EconomicOrientation', 'NewsConsumption', 'Ethnicity', 'Religiosity', 'PoliticalOrientation','Fluency']

for col in cols_to_update:
    regression_df_accuracy[col] = regression_df_accuracy[col] - 1


In [33]:
regression_df_accuracy.describe()

,fake_news_generation_source,PerceivedVeracity,Time_to_respond,WillingnessToShare,VaccinationStatus,SocialOrientation,EconomicOrientation,PoliticalOrientation,Concern,NewsConsumption,...,Religiosity,Income,Fluency,EducationYears,Ethnicity,StatementSequence,EthnicityBinary,GenderBinary,fall_for_fake_news,statement_source
count,19760.000000,19760.000000,19760.000000,19760.000000,19760.000000,19760.000000,19760.000000,19760.000000,19760.000000,19760.000000,...,19760.000000,19760.000000,19760.000000,19760.000000,19760.000000,19760.000000,19760.00000,19760.000000,19760.000000,19760.000000
mean,0.500000,1.229808,7.178595,0.012196,0.744939,1.910931,2.073887,1.173077,63.145749,3.028340,...,0.475709,0.982794,-0.012146,13.672065,0.676113,9.500000,0.79251,0.613360,0.429656,2.000000
std,0.500013,1.281389,11.607377,0.472218,0.435906,1.210713,1.200404,0.785390,33.018176,1.962554,...,0.605671,0.773248,0.109539,4.220952,1.666682,5.766427,0.40552,0.526924,0.495039,1.183246
min,0.000000,0.000000,2.004500,-1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,-1.000000,0.000000,0.000000,0.000000,0.00000,-1.000000,0.000000,0.000000
25%,0.000000,0.000000,3.708500,0.000000,0.000000,1.000000,1.000000,1.000000,35.000000,2.000000,...,0.000000,0.000000,0.000000,12.000000,0.000000,4.750000,1.00000,0.000000,0.000000,1.000000
50%,0.500000,1.000000,5.124250,0.000000,1.000000,2.000000,2.000000,1.000000,73.000000,3.000000,...,0.000000,1.000000,0.000000,14.000000,0.000000,9.500000,1.00000,1.000000,0.000000,2.500000
75%,1.000000,2.000000,7.465500,0.000000,1.000000,3.000000,3.000000,2.000000,91.000000,6.000000,...,1.000000,2.000000,0.000000,16.000000,0.000000,14.250000,1.00000,1.000000,1.000000,3.000000
max,1.000000,4.000000,476.294500,1.000000,1.000000,4.000000,4.000000,3.000000,100.000000,6.000000,...,2.000000,2.000000,0.000000,20.000000,7.000000,19.000000,1.00000,1.000000,1.000000,3.000000


In [34]:
regression_df_accuracy

,ResponseId,Statement,fake_news_generation_source,PerceivedVeracity,Time_to_respond,WillingnessToShare,VaccinationStatus,SocialOrientation,EconomicOrientation,PoliticalOrientation,...,Religiosity,Income,Fluency,EducationYears,Ethnicity,StatementSequence,EthnicityBinary,GenderBinary,fall_for_fake_news,statement_source
0,R_0N8NtfCM1SRFOtH,Hum01,0,0,3.3420,0,1,0,0,0,...,1,1,0,15,0,7,1,1,0,0
1,R_0N8NtfCM1SRFOtH,Hum02,0,2,6.5025,0,1,0,0,0,...,1,1,0,15,0,13,1,1,1,0
2,R_0N8NtfCM1SRFOtH,Hum03,0,0,2.7935,0,1,0,0,0,...,1,1,0,15,0,18,1,1,0,0
3,R_0N8NtfCM1SRFOtH,Hum04,0,1,6.1750,0,1,0,0,0,...,1,1,0,15,0,11,1,1,0,0
4,R_0N8NtfCM1SRFOtH,Hum05,0,0,2.6790,0,1,0,0,0,...,1,1,0,15,0,17,1,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19755,R_0C00kf5ULNIwGPv,AI06,1,0,2.9300,0,1,4,4,2,...,0,1,0,20,0,13,1,1,0,3
19756,R_0C00kf5ULNIwGPv,AI07,1,1,4.0460,0,1,4,4,2,...,0,1,0,20,0,12,1,1,0,3
19757,R_0C00kf5ULNIwGPv,AI08,1,1,9.8465,0,1,4,4,2,...,0,1,0,20,0,0,1,1,0,3
19758,R_0C00kf5ULNIwGPv,AI09,1,0,5.0180,0,1,4,4,2,...,0,1,0,20,0,5,1,1,0,3


In [35]:
# Correct the values for cehking the covid, on the numeric output from the server it is recorded 7 instead of 5
regression_df_accuracy['NewsConsumption'] = regression_df_accuracy['NewsConsumption'].replace(6,4)
regression_df_accuracy['NewsConsumption'].value_counts()

3    6120
4    5080
2    4180
1    2640
0    1740
Name: NewsConsumption, dtype: int64

In [36]:
regression_df_accuracy.to_csv(data_dir + 'processed_data.csv')